# Miara cosinusowa

In [1]:
import numpy as np            
def cosine(vec1, vec2):
    top = 0
    bot_a = 0
    bot_b = 0
    for i in range(len(vec1)):
        top += vec1[i] * vec2[i]
        bot_a += vec1[i] * vec1[i]
        bot_b += vec2[i] * vec2[i]
    bot = np.sqrt(bot_a) * np.sqrt(bot_b)
    return top/bot


print(cosine([1.0, 2.0, 3.0], [1.5, -0.7, -20]))
print(cosine([-10.0, 17.0, 2.0], [5.3, 12.0, -20]))
print(cosine([1.0, 2.0, 3.0], [1, -3000, 184]))
print(cosine([1.0, 2.0, 3.0], [1, 2, 3]))

-0.7977198918178166
0.23409628697705598
-0.48434715333575534
1.0


# Wykorzystanie BoW do porównania dwóch podobnych tekstów

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

doc1 = "Ala ma kota"
doc2 = "Ala ma pięknego puszystego kota"

docs = [doc1, doc2]

X_train_counts = count_vect.fit_transform(docs).todense()

print("Dokumenty reprezentowane jako BOW") # dokumenty w wierszach, słowa w kolumnach
print(X_train_counts)
print("\nPodobieńśtwo dokumentów to")
print(cosine(X_train_counts[0].tolist()[0], X_train_counts[1].tolist()[0])) # zamiana macierzy 1 x n na listę elementów 1 x n


Dokumenty reprezentowane jako BOW
[[1 1 1 0 0]
 [1 1 1 1 1]]

Podobieńśtwo dokumentów to
0.7745966692414834


# Wykorzystanie BoW do porównania synonimów

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

doc1 = "kot"
doc2 = "kotek"
docs = [doc1, doc2]


print("Dokumenty reprezentowane jako BOW") # dokumenty w wierszach, słowa w kolumnach
X_train_counts = count_vect.fit_transform(docs).todense()
print(X_train_counts)

print("\n\nPodobieńśtwo dokumentów to")
print(cosine(X_train_counts[0].tolist()[0], X_train_counts[1].tolist()[0]))

Dokumenty reprezentowane jako BOW
[[1 0]
 [0 1]]


Podobieńśtwo dokumentów to
0.0


# Embedding przykład

In [4]:
import numpy as np

def load_embeddings(path):
    mapping = dict()
    
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            line = line.strip()
            if len(line) == 0:
                continue
            splitted = line.split(" ")
            mapping[splitted[0]] = np.array(splitted[1:], dtype=float) # stwórz słownik słowo -> wektor 
    return mapping
    
# https://www.kaggle.com/datasets/watts2/glove6b50dtxt link do pliku
mapping = load_embeddings('glove.6B.50d.txt') # załadowanie wektora o długości 50 liczb

kot = mapping['cat']
kotek = mapping['kitten']
krzeslo = mapping['chair']

print("Podobieńśtwo między kotem a kotkiem:")
print(cosine(kot, kotek))

print("Podobieńśtwo między kotem a krzesłem")
print(cosine(kot, krzeslo))

Podobieńśtwo między kotem a kotkiem:
0.6386305647068644
Podobieńśtwo między kotem a krzesłem
0.2942529771662456


# Relacje między wektorami

In [5]:
def get_most_similar(vec1, embeddings): # funkcja zwracająca słowo leżące najbliżej danego wektora
    slowo = ""
    score = 0
    for word, vector in embeddings.items():
        if cosine(vec1, vector) > score:
            slowo = word
            score = cosine(vec1, vector)
    return slowo

new_point = mapping['italy'] - mapping['rome'] + mapping['warsaw']
print(get_most_similar(new_point, mapping))


poland


# Wykorzystanie embeddingów w klasyfikacji

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas
import numpy as np
from nltk import word_tokenize
from sklearn.metrics import classification_report

full_dataset = pandas.read_csv('spam_emails.csv', encoding='utf-8')      # wczytanie danych z pliku CSV
full_dataset['label_num'] = full_dataset.label.map({'ham':0, 'spam':1})  # mapowanie wartości na liczby 

np.random.seed(0)                                       # ustawienie seedu na 0
train_indices = np.random.rand(len(full_dataset)) < 0.7 # wylosowanie 70% wierszy, które znajdą się w zbiorze treningowym

train = full_dataset[train_indices] # wybór zbioru treningowego
test = full_dataset[~train_indices] # wybór zbioru testowego


def avg_embedding(doc, embedding):
    array = [embedding[token] for token in word_tokenize(doc.lower()) if token in embedding]
    return np.mean(array, axis=0)

def documents_to_ave_embeddings(docs, emb):
    return np.array([avg_embedding(doc, emb) for doc in docs])
    
 
classifier = SVC(C=1.0)

train_transformed = documents_to_ave_embeddings(train['text'], mapping)
print(train_transformed.shape)
test_transformed = documents_to_ave_embeddings(test['text'], mapping)


classifier.fit(train_transformed, train['label_num']) # wektoryzacja danych i trening klasyfikatora

# ------------------- OCENA KLASYFIKATORA -----------
accuracy = classifier.score(test_transformed, test['label_num'])
print(f"W zbiorze testowym poprawnie zaklasyfikowanych zostało {100*accuracy}% przypadków")
print(classification_report(test['label_num'], classifier.predict(test_transformed))) # testowanie klasyfikatora

(1624, 50)
W zbiorze testowym poprawnie zaklasyfikowanych zostało 91.81446111869032% przypadków
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       517
           1       0.88      0.84      0.86       216

    accuracy                           0.92       733
   macro avg       0.91      0.89      0.90       733
weighted avg       0.92      0.92      0.92       733



# Wykorzystanie odległości Levensteina oraz embeddingów do poprawienia literówek

In [8]:
from nltk.metrics.distance import edit_distance
from nltk import word_tokenize
from timeit import default_timer as timer

input_text = "Czerny kąt z bialym psem chodza rasem po dahu."

with open('slowa_min.txt', 'r', encoding='utf8') as f:
    valid_words = set([w.strip() for w in f.read().split("\n") if w.strip() != '']) # zbiór poprawnych słów

def get_closest_word(token, valid_words):
    slowo = ''
    score = 100
    for word in valid_words:
        if edit_distance(token, word) < score:
            slowo = word
            score = edit_distance(token, word)
    return slowo
        

tokenized_input = word_tokenize(input_text) # tokenizacja
start = timer()
corrected = []   # lista poprawionych tokenów
for token in tokenized_input:
    if not token.isalpha() or token in valid_words: # jeśli token nie jest słowem lub jest już w słowniku - nie poprawiaj
        corrected.append(token)
    else: # poprawianie
        corrected.append(get_closest_word(token, valid_words))
end = timer()


print("Tekst oryginalny: {t}".format(t=input_text))
print("Tekst poprawiony: {t}".format(t=" ".join(corrected)))
print("Czas poprawiania: {t}.".format(t=end-start))

Tekst oryginalny: Czerny kąt z bialym psem chodza rasem po dahu.
Tekst poprawiony: czarny kot za białym psem chodzą razem po dachu .
Czas poprawiania: 1.0332718999998178.
